In [3]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [4]:
import os
assert("compas-scores-raw.csv" in os.listdir())

columns =  ['Agency_Text', 'Sex_Code_Text', 'Ethnic_Code_Text',
       'DateOfBirth', 'ScaleSet_ID', 'AssessmentReason',
       'Language', 'LegalStatus', 'CustodyStatus', 'MaritalStatus',
       'RecSupervisionLevel', 'RecSupervisionLevelText',
       'Scale_ID', 'DisplayText', 'RawScore', 'DecileScore', 'ScoreText',
       'AssessmentType']

csv = pd.read_csv("compas-scores-raw.csv", usecols=columns)

In [ ]:
np.random.seed(40)
risk_violence = csv[csv["DisplayText"] == "Risk of Violence"]
risk_recidivism = csv[csv["DisplayText"] == "Risk of Recidivism"]
risk_failure_to_appear = csv[csv["DisplayText"] == "Risk of Failure to Appear"]

risk_recidivism